# 4. 토큰화 Tokenizing

## 4-a. 토큰화 계획
- word extraction 한 단어 리스트로 사용자 정의 사전을 만든다
- mecab 이용하여 tokenizing 한다

## 4-b. 토큰화 프로세스
1. mecab 을 컴파일하고, 테스트한다
  - 테스트할 text 를 제작한다
  - 토큰화 테스트
2. word extraction의 결과 단어 리스트를 불러온다
3. 사용자 정의 사전을 제작하고, .csv 파일로 저장해둔다
  - 종성이 있는지 판단한다
4. 여기까지 함수화한다 (단어 리스트 불러오기 + 사용자 정의 사전 파일로 저장)
5. 사용자 정의 사전을 `C:\mecab\user-dic` 에 옮긴다
6. 사용자 정의 사전을 컴파일한다
7. 토큰화 한다
  - [1-a] 의 text로 테스트를 진행한다
  - 변화가 있는지 체크한다

## 4-c. 사용자 정의사전 만들기

### 4-c-1. mecab 컴파일
- Mecab 메뉴얼 : https://kain7f1.notion.site/Mecab-195acbade3f14463b137c4490baebc76?pvs=4

### 4-c-2. (pycharm으로 실행) macab 토큰화 테스트

In [4]:
# 메뉴얼에 따라 빌드하고, pycharm에서 테스트합니다
import MeCab

txt1 = "주식으로 선거때마다 한탕 해처먹는 간잽이진짜 확망했으면"
m = MeCab.Tagger()
tokens = m.parse(txt1)
print(tokens)

ModuleNotFoundError: No module named 'MeCab'

### 4-c-3. word extraction의 결과 단어 리스트를 불러오기

In [7]:
# 텍스트 파일을 열고 읽기
file_path = "3_annlab_word_list.txt"
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# 줄바꿈 문자 제거
word_list = [line.strip() for line in lines]

# 결과 출력
print(word_list)

['엄청', '질문', '누구', '월에', '나도', '된다', '몰빵', '까지', '떡락', '지금', '등등', '학교', '미래', '구성', '팔아', '그래', '발표', '라는', '책임', '똑같', '리더', '반대', '영향', '뛰어', '다음', 'ㅅㅂ', '관련', '발행', '기사', '결정', '어떤', '저거', '경우', '선언', '본다', '올해', '지난', '애들', 'ㅇㅇ', '같이', '삼전', '국힘', '외국', '에서', '크게', '먼저', '포기', '팩트', '투자', '보수', '최소', '고점', '최대', '단타', '제발', '몰라', 'ㄷㄷ', '너무', '수도', '대표', '먹고', '요즘', '여러', '미친', '당선', '철수', '역시', '작년', '세계', '연락', '감사', '신고', '차트', '경영', '확인', '중국', '존버', '졸업', '느낌', '준비', '정치', '총선', '만원', '시간', '개미', '언제', '미국', '원래', '가장', '직접', '이번', '우리', '하나', '이런', '추천', '호남', '직원', '셀트', '일단', 'ㅠㅠ', '부분', '의원', '상장', '국내', '백신', '방법', '본인', '젊은', '허허', '다른', '벤처', '간다', '없음', '의견', '추가', '친구', '재산', '경쟁', '지지', '좋아', '에이', '저런', '시장', '폭등', '자신', '현대', '밖에', '게임', '인간', '총리', '좋은', '일부', '그냥', '후보', '시작', '전부', '관심', '양보', '함락', '오늘', '처음', '분명', '되어', '보안', '때문', '뭐냐', '의사', '최근', '투표', '기술', '가치', '시총', '노조', '상승', '경제', '모든', '내가', '호재', '재료', '같다', '시절', '서울', '위해', '갤주', '발언', '동기', '언론

In [8]:
len(word_list)

504

### 4-c-4. 사용자 정의 사전 만들기

In [9]:
# 단어의 마지막 글자에 종성이 있는지 체크하는 함수
def has_jongseong(char):
    if '가' <= char <= '힣':
        unicode_value = ord(char)
        jongseong = (unicode_value - 0xAC00) % 28
        return jongseong != 0
    else:
        return False

In [12]:
import numpy as np
import pandas as pd

data = []  # 사전에 들어갈 형태로 가공해서 넣을거임
for word in word_list:
    if len(word) == 0:
        continue
    if has_jongseong(word[-1]):  # 마지막 글자에 종성이 있으면 'T' 없으면 'F'
        jongseong = 'T'
    else:
        jongseong = 'F'
    # 사전 파일인 nnp.csv에 맞는 형식으로 데이터를 세팅함
    new_row = [word, np.nan, np.nan, 0, 'NNP', '*', jongseong, word, '*', '*', '*', '*']
    data.append(new_row)

df = pd.DataFrame(data)

fpath_to_save = 'user_dict_annlab.csv'
df.to_csv(fpath_to_save, index=False, encoding="utf-8", header=False)
print(f'{fpath_to_save} 파일이 생성되었습니다')

user_dict_annlab.csv 파일이 생성되었습니다


### 4-c-5. 사용자 정의 사전 제작 함수화

In [13]:
import numpy as np
import pandas as pd

def make_user_dict(fpath_word_list, fpath_user_dict):
    # 1. word_list 파일 읽어오기
    print("[progress 1/3] word_list 파일 읽어오기")
    with open(fpath_word_list, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    word_list = [line.strip() for line in lines]  # 줄바꿈 문자 제거

    # 단어의 마지막 글자에 종성이 있는지 체크하는 함수
    def has_jongseong(char):
        if '가' <= char <= '힣':
            unicode_value = ord(char)
            jongseong = (unicode_value - 0xAC00) % 28
            return jongseong != 0
        else:
            return False

    # 2. mecab 사전의 형식으로 데이터 가공하기
    print("[progress 2/3] mecab 사전의 형식으로 데이터 가공하기")
    data = []  # 사전에 들어갈 형태로 가공해서 넣을거임
    for word in word_list:
        if len(word) == 0:
            continue
        if has_jongseong(word[-1]):  # 마지막 글자에 종성이 있으면 'T' 없으면 'F'
            jongseong = 'T'
        else:
            jongseong = 'F'
        # 사전 파일인 nnp.csv에 맞는 형식으로 데이터를 세팅함
        new_row = [word, np.nan, np.nan, 0, 'NNP', '*', jongseong, word, '*', '*', '*', '*']
        data.append(new_row)
    
    df = pd.DataFrame(data)

    # 3. 가공한 데이터를 파일로 저장하기
    print("[progress 3/3] 가공한 데이터를 파일로 저장하기")
    df.to_csv(fpath_user_dict, index=False, encoding="utf-8", header=False)
    print(f"{fpath_user_dict} 파일이 생성되었습니다")

    print("[done]")

### 4-c-6. 함수 테스트

In [15]:
fpath_word_list = "3_annlab_word_list.txt"
fpath_user_dict = "user_dict_annlab.csv"

make_user_dict(fpath_word_list, fpath_user_dict)

[progress 1/3] word_list 파일 읽어오기
[progress 2/3] mecab 사전의 형식으로 데이터 가공하기
[progress 3/3] 가공한 데이터를 파일로 저장하기
user_dict_annlab.csv 파일이 생성되었습니다
[done]


## 4-d. 사용자 정의 사전 컴파일
### 4-d-1. 사용자 정의 사전을 C:\mecab\user-dic 에 옮긴다
### 4-d-2. 사용자 정의 사전을 컴파일한다 (Mecab 메뉴얼 참고)

## 4-e. 토큰화 Tokenizing